# Welcome back Commander!

This is part 4, see parts [1](https://www.kaggle.com/code/bovard/kore-fleets-intro-part-1-the-basics) (basic info), [2](https://www.kaggle.com/bovard/kore-intro-part-2-mining-kore) (mining), and [3](https://www.kaggle.com/bovard/kore-intro-part-3-expanding-the-empire) (expanding) if interested!

We will take a look at combat here!

In [ ]:
!pip install kaggle-environments

In [ ]:
from kaggle_environments import make
env = make("kore_fleets", debug=True)
print(env.name, env.version)

## Fleet to fleet combat

When two fleets move to occupy the same tile or move orthogonally adjacent to an enemy fleet they do damage to eachother!

#### Same tile
The largest fleet (in ship count) wins, surviving with `intial_ships - second_largest_fleet_ships` ships left. For example, if fleets from 3 different players (1, 2, 3) enter the same tile with f1 having 100 ships, f2 having 50, and f3 having 25, f1 would survive with 100 - 50 = 50 ships. The surviving fleet, if there is one, takes the kore from all the other fleets.

#### Adjacent tiles
After a fleet moves and resolves and on-tile collisions, it does damage to all adjacent tiles equal to its ship count. For example, if two fleets of different players of sizes 100 and 50 end up adjacent, the 100 ship fleet will survive with only 50 ships left. Note that a single fleet can do damage to up to 3 other fleets if positioned correctly!

Let's see it in action! With the [starter bot](https://www.kaggle.com/code/bovard/basic-python-starter-notebook).

In [ ]:
%%writefile starter.py
# from https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/kore_fleets/starter_bots/python/main.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore

    for shipyard in me.shipyards:
        if shipyard.ship_count > 10:
            direction = Direction.from_index(turn % 4)
            action = ShipyardAction.launch_fleet_with_flight_plan(randint(2,3), direction.to_char())
            shipyard.next_action = action
        elif kore_left > spawn_cost * shipyard.max_spawn:
            action = ShipyardAction.spawn_ships(shipyard.max_spawn)
            shipyard.next_action = action
            kore_left -= spawn_cost * shipyard.max_spawn
        elif kore_left > spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            shipyard.next_action = action
            kore_left -= spawn_cost

    return me.next_actions

In [ ]:
starter_agent_path = "/kaggle/working/starter.py"
env.run([starter_agent_path, starter_agent_path, starter_agent_path, starter_agent_path])
env.render(mode="ipython", width=1000, height=800)

As you can see above, there is a lot of collisions and adjacent damage.

## Capturing a Shipyard

In addition to collisions, you can also capture an enemy shipyard. The same mechanics are used in collision - the larger fleet wins.

When you capture a shipyard the ship production rate of that shipyard is reset to 1!

Let's write a bot that amasses 50 ships and attempts to take over nearby shipyards. Note that we'll change our spawn code to take advantage of a shipyards increased production rate `shipyard.max_spawn`! This will allow us to spawn up to 10 ships a turn if we hold onto the shipyard long enough

In [ ]:
%%writefile attacker.py
# from https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/kore_fleets/starter_bots/python/main.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def get_closest_enemy_shipyard(board, position, me):
    min_dist = 1000000
    enemy_shipyard = None
    for shipyard in board.shipyards.values():
        if shipyard.player_id == me.id:
            continue
        dist = position.distance_to(shipyard.position, board.configuration.size)
        if dist < min_dist:
            min_dist = dist
            enemy_shipyard = shipyard
    return enemy_shipyard


def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore

    for shipyard in me.shipyards:
        action = None
        if turn % 100 < 20 and shipyard.ship_count >= 50:
            closest_enemy_shipyard = get_closest_enemy_shipyard(board, shipyard.position, me)
            if not closest_enemy_shipyard:
                continue
            enemy_pos = closest_enemy_shipyard.position
            my_pos = shipyard.position
            flight_plan = "N" if enemy_pos.y > my_pos.y else "S"
            flight_plan += str(abs(enemy_pos.y - my_pos.y) - 1)
            flight_plan += "W" if enemy_pos.x < my_pos.x else "E"
            action = ShipyardAction.launch_fleet_with_flight_plan(shipyard.ship_count, flight_plan)            
        elif shipyard.ship_count >= 10 and turn % 7 == 0 and turn % 100 > 20 and turn % 100 < 90:
            direction = Direction.from_index(turn % 4)
            opposite = direction.opposite()
            flight_plan = direction.to_char() + "9" + opposite.to_char()
            action = ShipyardAction.launch_fleet_with_flight_plan(10, flight_plan)
        elif kore_left > spawn_cost * shipyard.max_spawn:
            action = ShipyardAction.spawn_ships(shipyard.max_spawn)
            kore_left -= spawn_cost * shipyard.max_spawn
        elif kore_left > spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            kore_left -= spawn_cost
        shipyard.next_action = action

    return me.next_actions

In [ ]:
starter_agent_path = "/kaggle/working/starter.py"
attacker_agent_path = "/kaggle/working/attacker.py"
env.run([attacker_agent_path, starter_agent_path, starter_agent_path, starter_agent_path])
env.render(mode="ipython", width=1000, height=800)

### Victory! 

Notice the number in the bottom right hand corner of the shipyard after you take it over (which displays `shipyard.max_spawn`) resetting to 1 and slowly incrementing when you take over a shipyard.

Good luck commander, and happy coding!